In [ ]:
# !pip install mlxtend==0.17.2
# ## run when get error <ImportError: cannot import name 'fpgrowth'>: target version 0.17.2
!pip install --upgrade mlxtend
!pip show mlxtend

     |████████████████████████████████| 1.4MB 5.7MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
Name: mlxtend
Version: 0.18.0
Summary: Machine Learning Library Extensions
Home-page: https://github.com/rasbt/mlxtend
Author: Sebastian Raschka
Author-email: mail@sebastianraschka.com
License: BSD 3-Clause
Location: /usr/local/lib/python3.6/dist-packages
Requires: matplotlib, pandas, joblib, scikit-learn, scipy, setuptools, numpy
Required-by: 


In [ ]:
# mount drive to notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import fpmax
import itertools
import copy
import numpy as np
from scipy.stats import stats
import random

import os
import sys

In [ ]:
def pause(pause_message):
    programPause = input('Error, Error, Press enter to exit')
    sys.exit(pause_message)
# while(True):
#   pause('Stop :(((((((((((( in pause() funtion')

In [ ]:
# convert transaction database to target form of model
from mlxtend.preprocessing import TransactionEncoder
def convert_transaction_to_df_one_hot_vector(_transaction_dataset, sparse_format = True):
  te = TransactionEncoder()
  if(sparse_format):
    # To save memory,represent your transaction data in the sparse format. This is especially useful if you have lots of products and small transactions.
    te_ary = te.fit(_transaction_dataset).transform(_transaction_dataset, sparse=True)
    #make one-hot-vector dataframe form
    return pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)
  # else:
  te_ary = te.fit(_transaction_dataset).transform(_transaction_dataset)
  return pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
# Save data to file by pickle
import pickle
def save_data(dataset, loc_file, pre_loc_file_url = '/content/drive/My Drive/DuLieuNhaHH102/model/'):
  loc_file = pre_loc_file_url + loc_file
  with open(loc_file, 'wb') as filehandle:
      # store the data as binary data stream
      pickle.dump(dataset, filehandle)
def load_data(loc_file, pre_loc_file_url = '/content/drive/My Drive/DuLieuNhaHH102/model/'):
  loc_file = pre_loc_file_url + loc_file
  with open(loc_file, 'rb') as filehandle:
      # read the data as binary data stream
      dataset = pickle.load(filehandle)
  return dataset

In [ ]:
#remove list of lists of it is an subset of another list in lists
def remove_subset(list_of_list):              
    sets={frozenset(e) for e in list_of_list}  
    us=[]
    while sets:
        e=sets.pop()
        if any(e.issubset(s) for s in sets) or any(e.issubset(s) for s in us):
            continue
        else:
            us.append(sorted(list(e)))   
    return us

In [ ]:
def print_list_with_minK(list_of_list, min_k_item):
  count = 0
  for item in list_of_list:
    if(len(item) >= min_k_item):
      count +=1
      print(item)
  print(f"Len -origin:{len(list_of_list)} ")
  print(f"Len -filter by min_k_item={min_k_item} :{count} ")

In [ ]:
# CONSTANT VAR
TIME_WINDOW_IN_MINUTES = 1
THRESHOLD_CV =0.9

In [ ]:
def find_groups_dev(df_transaction_dataset, min_threshold = 0.9, max_len_itemset = 10, algo = 0, _verbose = 1, _low_meomory = False):
  '''
  algo = 0 > using fpgrowth
  algo = 1 > using apriori
  algo = 3 > using fpmax
   `verbose` : int (default: 0)

    Shows the number of iterations if >= 1 and `low_memory` is `True`. If
    >=1 and `low_memory` is `False`, shows the number of combinations.


  - `low_memory` : bool (default: False)

      If `True`, uses an iterator to search for combinations above
      `min_support`.
      Note that while `low_memory=True` should only be used for large dataset
      if memory resources are limited, because this implementation is approx.
      3-6x slower than the default.

  '''
  items  = df_transaction_dataset.columns
  related_dev_groups = []

  #find itemset tight with each specific item
  for item in items:
    current_group = []

    # get all transaction that contain current {item} -> make *item-transaction
    data_extract = df_transaction_dataset[df_transaction_dataset[item] == True]
    if algo == 0:
      # using fpgrowth algorithm
      frequent_itemset = fpgrowth(data_extract, min_support=min_threshold, use_colnames=True, max_len = max_len_itemset, verbose=_verbose)
    elif algo == 1:
      # using apriori algorithm
      frequent_itemset = apriori(data_extract, min_support=min_threshold, use_colnames=True ,max_len = max_len_itemset ,verbose=_verbose,low_memory = _low_meomory)

    else:
      # using fpmax:
      '''
      FP-Max is a variant of FP-Growth, which focuses on obtaining maximal itemsets.
      An itemset X is said to maximal if X is frequent and there exists no frequent super-pattern containing X.
      In other words, a frequent pattern X cannot be sub-pattern of larger frequent pattern to qualify for the definition maximal itemset.
      '''
      frequent_itemset = fpmax(data_extract, min_support=min_threshold, use_colnames=True,max_len = max_len_itemset, verbose=_verbose)
    
    for x in frequent_itemset.itemsets:

      list_fre_items = sorted(list(x)) 
      
      if (item in x) & (len(x) > 1):
        current_group.append(list_fre_items)
    related_dev_groups = related_dev_groups + remove_subset(current_group)
  related_dev_groups.sort()
  related_dev_groups = list(related_dev_groups for related_dev_groups,_ in itertools.groupby(related_dev_groups))
  return related_dev_groups

In [ ]:
# function to apply make is_binary_device col in data_preprocessing()
def is_binary_device_by_df(df):
    if (df['device_id'][0:1] == 'M' 
        or df['device_id'][0:1] == 'D' 
        or df['device_id'][0:2] == 'L0'):
        # or df['device_id'][0:2] == 'BA'): #????????
        val = bool(True)
    else:
        val = bool(False)
    return val
    # if len(df['device_id'].device_value.unique()) == 2 
    #     val = bool(True)
    # else:
    #     val = bool(False)
    # return val
 
def is_binary_device_by_id(device_id):
    if (device_id[0:1] == 'M' 
        or device_id[0:1] == 'D' 
        or device_id[0:2] == 'L0'):
        val = bool(True)
    else:
        val = bool(False)
    return val
    # if len(data_raw[data_raw.device_id == device_id].device_value.unique()) == 2 
    #     val = bool(True)
    # else:
    #     val = bool(False)
    # return val

In [ ]:
def data_preprocessing(data):

# convert string binary value to 0,1
  data['device_value'] =  data['device_value'].replace(['ON', 'OPEN'], 1)
  data['device_value'] =  data['device_value'].replace(['OFF', 'CLOSE'], 0)

# convert numeric in str value to numeric value
  data.device_value = pd.to_numeric(data.device_value, errors='raise')

# create collumn is_binary_device
  data['is_binary_device'] = data.apply(is_binary_device_by_df, axis=1)

# GET DATA WITHOUT BATTERY LEVEL device
  data.drop(data[data['device_id'].astype(str).str[0:2] == 'BA'].index, axis =0, inplace = True)

  data.drop(['ann'], axis=1, inplace =True)

  items = data['device_id'].unique()
  print(f'length of items: {len(items)}, items: {items}')
  print(data.head())
  print(data.info())

In [ ]:
# filename = 'twor.2010/twor.2010/format_data.txt'
filename = '/content/drive/My Drive/DuLieuNhaHH102/ann_format.txt'
data_raw = pd.read_csv(filename, sep =' ', names = ["date", "time", "device_id", "device_value", "ann"],
                       parse_dates ={'datetime': ['date', 'time']} )
# filename = 'hh102/ann_format.txt'
# data_raw = pd.read_csv(filename, sep =' ', names = ["date", "time", "device_id", "device_value","activity"],
#                        parse_dates ={'datetime': ['date', 'time']} )

In [ ]:
# data = data_raw.copy(deep=True)
# data_preprocessing(data)

In [ ]:
# save_data(data, 'ann_data.data')
data = load_data('ann_data.data')
data

,datetime,device_id,device_value,is_binary_device
0,2011-06-15 00:06:32.834414,M021,1,True
1,2011-06-15 00:06:33.988964,M021,0,True
3,2011-06-15 00:15:01.957718,LS013,6,False
4,2011-06-15 00:25:01.892474,LS013,7,False
16,2011-06-15 03:37:46.585185,M021,1,True
...,...,...,...,...
413130,2011-08-15 22:44:24.343584,LS020,23,False
413132,2011-08-15 22:54:24.303897,LS020,22,False
413133,2011-08-15 22:54:25.727398,LS021,17,False
413134,2011-08-15 23:04:25.687194,LS021,18,False


In [ ]:
# IQR
# calculate interquartile range
list_numeric_device = list(data[data['is_binary_device'] == False].device_id.unique()).copy()
iqr_column_names = ['device_id', 'q25', 'q75', 'lower', 'upper']
df_iqr = pd.DataFrame(columns = iqr_column_names)
for dev_id in list_numeric_device:
  data_by_device = data[data['device_id'] == dev_id]
  _mean = data_by_device.device_value.mean()
  _max = data_by_device.device_value.max()
  _min = data_by_device.device_value.min()# ( cai nay co the tinh duoc +- alpha*std)
  _std = data_by_device.device_value.std()
  q25, q75 = data_by_device.device_value.quantile(.25), data_by_device.device_value.quantile(.75)
  iqr = q75 - q25
  cut_off = iqr * 1.5
  lower, upper = q25 - cut_off, q75 + cut_off
  uni_cut_off = 3*_std
  uni_lower, uni_upper = _mean - uni_cut_off, _mean + uni_cut_off 
  df_iqr = df_iqr.append({'device_id': dev_id, '_mean': _mean, 'std': _std, 'uni_lower': uni_lower, 'uni_upper': uni_upper,
                          'q25': q25, 'q75': q75, 'lower': lower, 'upper': upper, '_min': _min, '_max': _max}, ignore_index=True)

In [ ]:
def trenddetector(list_seq_of_data, order=1):
    '''
    need >= 3 values to detect exact trend
    '''
    arr_of_index = np.arange(len(list_seq_of_data))
    # if len = 3 -> arr_of_index = 0,1,2
    arr_seq_of_data = np.array(list_seq_of_data)
    result = np.polyfit(arr_of_index, arr_seq_of_data, order)
    slope = result[-2]
    # return float(slope)
    return round(float(slope),1)
# need >=3 v
# trenddetector([3,5,3]), trenddetector([3,4,3]) == 0

In [ ]:
np.seterr('raise') # raise error instead of only warning
def coefficient_of_variation(list_seq_of_data):
  '''
  need >= 3 values to detect exact CV
  '''
  mean_of_list = sum(list_seq_of_data) / len(list_seq_of_data)
  if mean_of_list == 0:
    # avoid mean error: FloatingPointError: invalid value encountered in double_scalars WHEN cal CV
   return 0
  arr_seq_of_data = np.array(list_seq_of_data)
  CV = stats.variation(arr_seq_of_data)
  return CV
# coefficient_of_variation([1,25,1])
# coefficient_of_variation([0,0,0])
# coefficient_of_variation([1,25,1,1])
# coefficient_of_variation([1,25,1,25]) 0.9230769230769231 -> co the phai dat threshold la 0.9
# coefficient_of_variation([1,25,1,1]), coefficient_of_variation([25,1,1])
# coefficient_of_variation([1,25,1])
# coefficient_of_variation([1,25,20,15])
# coefficient_of_variation([1,25,20]) 0.6742635834465576
# coefficient_of_variation([1,25,1,1 ]) 1.4846149779161806
# co ve de ro nhat spike thi phai gen kieu 1 1 25 1 1 hay it nhat la 1 25 1 1 KHAC kieu high variance la 1 25 1 25 1..

# neu khong hieu qua ->  khong bit trend co du de detect ko hay phai doi CV sang chi so skewness

In [ ]:
# # data.datetime.iloc[0] == Timestamp('2011-06-15 00:06:32.834414')
# # start_date = datetime.datetime(2011, 6, 16) # 2011-06-16 00:00:00

# Gen transaction dataset
transaction_dataset = []

# number_of_days = 13
# number_of_hours = 300 # ~ 12.5 days
number_of_hours = 14*24 # ~ 2 weeks


start_date = data.iloc[0].datetime
last_datetime =  start_date + datetime.timedelta(hours=number_of_hours)
print(start_date, last_datetime)

data_to_make_group_dev = data[(data['datetime'] < last_datetime)].copy(deep=True)
start = copy.deepcopy(start_date)

# while start<last_datetime:
#     end = start + datetime.timedelta(minutes=(TIME_WINDOW_IN_MINUTES))
#     data_extract = data_to_make_group_dev[(data_to_make_group_dev['datetime'] >= start) & (data_to_make_group_dev['datetime'] < end)]
#     # print(data_extract)
#     if not data_extract.empty:
#       current_items = list(data_extract.device_id.unique())
#       num_of_items = len(current_items)
#       print(num_of_items,start,end,end-start)
#       if(num_of_items>1):
#         # num_window +=1
#         # print(f'number of window: {num_window}')
#         transaction_dataset.append(current_items)
        
#     start = end


2011-06-15 00:06:32.834414 2011-06-29 00:06:32.834414


In [ ]:
# group_dev = []
# # group_dev = find_groups_dev(df_transaction_dataset.head(13980),0.6, 20, algo=3, _verbose=0)
# group_dev = find_groups_dev(df_transaction_dataset,0.6, 10, algo=3, _verbose=0)

# print(f'origin: {len(group_dev)}')
# # print(group_dev)
# # print_list_with_minK(group_dev,2)

# group_dev = remove_subset(group_dev)
# len_group_dev =  len(group_dev)
# print(f'after remove subset:{len_group_dev} ')
# # print_list_with_minK(group_dev,2)

origin: 69
after remove subset:37 


In [ ]:
# save_data(transaction_dataset, 'ann_transaction_dataset_14days.data')
# save_data(group_dev, 'ann_group_dev_14days.data')

In [ ]:
transaction_dataset = load_data('ann_transaction_dataset_14days.data')
group_dev = load_data('ann_group_dev_14days.data')

In [ ]:
df_transaction_dataset = convert_transaction_to_df_one_hot_vector(transaction_dataset)

In [ ]:
print_list_with_minK(group_dev,2)

['D002', 'LS003', 'M001', 'M002', 'MA003']
['LS014', 'M015', 'M016', 'MA014']
['M021', 'M022', 'MA020']
['LS011', 'LS012']
['LS014', 'M011', 'M012', 'MA010', 'MA014']
['LS016', 'M016']
['LS006', 'M006']
['LS007', 'LS008']
['D005', 'LS007', 'M007', 'M008']
['L001', 'LL001', 'LS001', 'LS003', 'M001', 'M002', 'MA003', 'MA009']
['LS002', 'M002', 'MA003']
['D006', 'LS007', 'M007', 'M008']
['L004', 'LS014', 'LS017', 'LS018', 'M017', 'M018', 'MA014']
['LS019', 'LS020']
['M005', 'M006', 'M007']
['LS004', 'M004']
['M011', 'M012', 'M017', 'M018', 'MA010', 'MA014']
['M019', 'M022', 'MA020']
['LS010', 'LS012']
['M019', 'MA020', 'MA023']
['LS005', 'LS006']
['LS010', 'MA010']
['L003', 'LS013', 'LS018', 'M018', 'MA013']
['L002', 'LS003', 'LS009', 'M002', 'MA003', 'MA009']
['LS003', 'M011', 'M012', 'MA003', 'MA010', 'MA014']
['D002', 'LS001', 'M001', 'M002', 'MA003']
['LS012', 'LS014']
['L005', 'LL005', 'LS018', 'M018', 'M019', 'MA020']
['D005', 'LS008', 'M007', 'M008']
['L003', 'LS013', 'M018', 'M019

In [ ]:
len_group_dev =  len(group_dev)
len_group_dev ,group_dev
set_devices_to_detect = set()
b = set(data_to_make_group_dev.device_id.unique())
for i in group_dev:
  set_devices_to_detect.update(i)
print(f'group_dev covers {len(set_devices_to_detect)} devs')
print(f'{len(b-set_devices_to_detect)} independent devs')
inp_devices = list(b-set_devices_to_detect)
print('independent devs: ',inp_devices)
SORTED_LIST_DEVICES_TO_DETECT = sorted(list(set_devices_to_detect))
# SORTED_LIST_DEVICES_TO_DETECT

group_dev covers 54 devs
7 independent devs
independent devs:  ['T104', 'T101', 'LS022', 'T103', 'T105', 'T102', 'LS021']


In [ ]:
for dev in SORTED_LIST_DEVICES_TO_DETECT:
  data.loc[data.device_id == dev, dev] = data.device_value
# data.loc[data.device_id == 'M021', 'M021'] = data.device_value

In [ ]:
data

,datetime,device_id,device_value,is_binary_device,D002,D005,D006,L001,L002,L003,L004,L005,LL001,LL005,LS001,LS002,LS003,LS004,LS005,LS006,LS007,LS008,LS009,LS010,LS011,LS012,LS013,LS014,LS015,LS016,LS017,LS018,LS019,LS020,LS023,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023
0,2011-06-15 00:06:32.834414,M021,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-06-15 00:06:33.988964,M021,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-06-15 00:15:01.957718,LS013,6,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-06-15 00:25:01.892474,LS013,7,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2011-06-15 03:37:46.585185,M021,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413130,2011-08-15 22:44:24.343584,LS020,23,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413132,2011-08-15 22:54:24.303897,LS020,22,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413133,2011-08-15 22:54:25.727398,LS021,17,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413134,2011-08-15 23:04:25.687194,LS021,18,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.ffill(inplace=True)
# data.head(30)

In [ ]:
data

,datetime,device_id,device_value,is_binary_device,D002,D005,D006,L001,L002,L003,L004,L005,LL001,LL005,LS001,LS002,LS003,LS004,LS005,LS006,LS007,LS008,LS009,LS010,LS011,LS012,LS013,LS014,LS015,LS016,LS017,LS018,LS019,LS020,LS023,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023
0,2011-06-15 00:06:32.834414,M021,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-06-15 00:06:33.988964,M021,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-06-15 00:15:01.957718,LS013,6,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-06-15 00:25:01.892474,LS013,7,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2011-06-15 03:37:46.585185,M021,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413130,2011-08-15 22:44:24.343584,LS020,23,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,16.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413132,2011-08-15 22:54:24.303897,LS020,22,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,16.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413133,2011-08-15 22:54:25.727398,LS021,17,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,16.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413134,2011-08-15 23:04:25.687194,LS021,18,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,16.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
index_first_time_all_device_has_valid_value = []
for col in SORTED_LIST_DEVICES_TO_DETECT:
  # print(data[col].first_valid_index())
  index_first_time_all_device_has_valid_value.append(data[col].first_valid_index())
print(SORTED_LIST_DEVICES_TO_DETECT)
print(index_first_time_all_device_has_valid_value)
print(max(index_first_time_all_device_has_valid_value))
# vi tri ma tat ca cac thiet bi ke tu vi tri nay tro ve truoc tat ca cac thiet bi deu duoc report it nhat mot lan (duoc ghi lai trong data_raw)
first_index_valid = max(index_first_time_all_device_has_valid_value)

['D002', 'D005', 'D006', 'L001', 'L002', 'L003', 'L004', 'L005', 'LL001', 'LL005', 'LS001', 'LS002', 'LS003', 'LS004', 'LS005', 'LS006', 'LS007', 'LS008', 'LS009', 'LS010', 'LS011', 'LS012', 'LS013', 'LS014', 'LS015', 'LS016', 'LS017', 'LS018', 'LS019', 'LS020', 'LS023', 'M001', 'M002', 'M004', 'M005', 'M006', 'M007', 'M008', 'M011', 'M012', 'M015', 'M016', 'M017', 'M018', 'M019', 'M021', 'M022', 'MA003', 'MA009', 'MA010', 'MA013', 'MA014', 'MA020', 'MA023']
[561, 1163, 6404, 83881, 2028, 881, 6853, 6823, 83880, 6822, 137, 126, 120, 122, 124, 118, 136, 133, 2029, 119, 143, 159, 3, 129, 125, 132, 158, 295, 259, 139, 3613, 558, 555, 1149, 1157, 1154, 1158, 1164, 546, 544, 986, 946, 533, 44, 41, 0, 634, 553, 2027, 547, 45, 536, 19, 3612]
83881


In [ ]:
data.loc[83881].device_value

1

In [ ]:
data.loc[83881].datetime

Timestamp('2011-06-26 20:51:28.252875')

In [ ]:
data_valuable = data[data.datetime >= data.loc[83881].datetime].copy(deep = True)
data_valuable.head(20)

,datetime,device_id,device_value,is_binary_device,D002,D005,D006,L001,L002,L003,L004,L005,LL001,LL005,LS001,LS002,LS003,LS004,LS005,LS006,LS007,LS008,LS009,LS010,LS011,LS012,LS013,LS014,LS015,LS016,LS017,LS018,LS019,LS020,LS023,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023
83881,2011-06-26 20:51:28.252875,L001,1,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83882,2011-06-26 20:51:28.427867,LL001,63,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83883,2011-06-26 20:51:28.929415,LS003,2,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83884,2011-06-26 20:51:28.959199,MA003,1,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
83885,2011-06-26 20:51:29.282708,LS001,7,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,7.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
83886,2011-06-26 20:51:29.339779,M001,1,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,7.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
83887,2011-06-26 20:51:31.899326,LS001,10,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,10.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
83888,2011-06-26 20:51:31.934922,M001,0,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,10.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
83889,2011-06-26 20:51:32.207508,MA009,1,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,10.0,0.0,2.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
83890,2011-06-26 20:51:33.410881,LS003,7,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,63.0,0.0,10.0,0.0,7.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_valuable[data_valuable.device_id == 'LS001'].head(50)
len(data_valuable[data_valuable.device_id == 'L001'])

26

In [ ]:
data[data.datetime < data.loc[83881].datetime].copy(deep = True).tail(1)
# # chi co duy nhat L001 chua co du lieu. De lam ban dau co the coi L001 (la mot cam bien nhi phan) co gia tri la 0 de tinh previous
#                     datetime	device_id	device_value	is_binary_device	D002	D005	D006	L001	L002	L003	L004	L005	LL001	LL005	LS001
# 83880	2011-06-26 20:51:28.227872	LL001	           50	False	             0.0	0.0	  0.0	   NaN	 0.0	 0.0	 0.0	 0.0	 50.0	  0.0	  0.0
# # Ta se fill L001 la 0

,datetime,device_id,device_value,is_binary_device,D002,D005,D006,L001,L002,L003,L004,L005,LL001,LL005,LS001,LS002,LS003,LS004,LS005,LS006,LS007,LS008,LS009,LS010,LS011,LS012,LS013,LS014,LS015,LS016,LS017,LS018,LS019,LS020,LS023,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023
83880,2011-06-26 20:51:28.227872,LL001,50,False,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.loc[83880, 'L001'] = 0

In [ ]:
data[data.datetime < data.loc[83881].datetime].copy(deep = True).tail(1)
# data_valuable[data_valuable.datetime < data_valuable.loc[83881].datetime].copy(deep = True).tail(1)

,datetime,device_id,device_value,is_binary_device,D002,D005,D006,L001,L002,L003,L004,L005,LL001,LL005,LS001,LS002,LS003,LS004,LS005,LS006,LS007,LS008,LS009,LS010,LS011,LS012,LS013,LS014,LS015,LS016,LS017,LS018,LS019,LS020,LS023,M001,M002,M004,M005,M006,M007,M008,M011,M012,M015,M016,M017,M018,M019,M021,M022,MA003,MA009,MA010,MA013,MA014,MA020,MA023
83880,2011-06-26 20:51:28.227872,LL001,50,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
keys = range(len_group_dev)
values = copy.deepcopy(group_dev)
dict_get_group_by_id = dict(zip(keys, values))
# print(dict_get_group_by_id.get(0))

keys = range(len_group_dev)
values =[[] for i in range(len_group_dev)]
dict_group_state = dict(zip(keys, values))

# init state of each group dev ( state when this group not exist in time_window) 
# ( key: group_d, value init state of this group with group id ->
# Type value: String: * value is a string -> use on collect transition state of each group)
dict_init_group_state = dict()

# print(dict_group_state)
# print(dict_get_group_by_id)
for key, value in dict_get_group_by_id.items():
  init_state = ''
  for device in value:
    if data[data.device_id == device].iloc[0].is_binary_device:
      init_state += '0'
      # print('INITSTATE', device, '0')
    else:
      init_state += '000'
      # print('INITSTATE', device, '000')

  dict_group_state[key].append(init_state)
  dict_init_group_state[key] = init_state

print("dict_get_group_by_id", dict_get_group_by_id)
print("dict_group_state", dict_group_state)
print("dict_init_group_state", dict_init_group_state)

dict_get_group_by_id {0: ['LS003', 'M011', 'M012', 'MA003', 'MA010', 'MA014'], 1: ['LS002', 'M002', 'MA003'], 2: ['LS015', 'M015', 'MA014'], 3: ['L001', 'LL001', 'LS001', 'LS003', 'M001', 'M002', 'MA003', 'MA009'], 4: ['LS019', 'LS020'], 5: ['D002', 'LS001', 'LS003', 'M001', 'MA003'], 6: ['LS014', 'M015', 'M016', 'MA014'], 7: ['L005', 'LL005', 'LS018', 'M018', 'M019', 'MA020'], 8: ['LS007', 'LS008'], 9: ['LS010', 'MA010'], 10: ['LS016', 'M016'], 11: ['D006', 'LS007', 'M007', 'M008'], 12: ['M021', 'M022', 'MA020'], 13: ['L002', 'LS003', 'LS009', 'M002', 'MA003', 'MA009'], 14: ['L004', 'LS014', 'LS017', 'LS018', 'M017', 'M018', 'MA014'], 15: ['L003', 'LS013', 'M018', 'M019', 'MA013', 'MA020'], 16: ['M005', 'M006', 'M007'], 17: ['M019', 'M022', 'MA020'], 18: ['L003', 'LS013', 'LS018', 'M018', 'MA013'], 19: ['LS023', 'MA020', 'MA023'], 20: ['LS010', 'LS012'], 21: ['D005', 'D006', 'M007', 'M008'], 22: ['LS014', 'M011', 'M012', 'MA010', 'MA014'], 23: ['M019', 'MA020', 'MA023'], 24: ['LS004',

In [ ]:
def get_current_state_of_each_dev(data_to_get, mileston_datetime):
  '''
  binary device: 1bit
  GET = OR all value:
  = 1 if OR = 1
  = 0 if OR = 0 ~ There are no changes in the state of THIS device | There is no change in ANY device state <=> There NO device report in current TIME_WINDOW


  numeric device: 3bit
  {CV>THRESHOLD_CV}{TREND>0}{TREND=0}

  {TREND>0}{TREND=0}: 10 -> Increase Trend
  {TREND>0}{TREND=0}: 00 -> Decrease Trend
  {TREND>0}{TREND=0}: 01 -> Stable Trend| No Trend | There are no changes in the state of THIS device| There is no change in ANY device state <=> There NO device report in current TIME_WINDOW
      NOT HAVE {TREND>0}{TREND=0}: 11
  
  Stable Trend {TREND>0}{TREND=0}: 01 => {CV>THRESHOLD_CV}{TREND>0}{TREND=0}: 001
  Decrease Trend => {CV>THRESHOLD_CV}{TREND>0}{TREND=0}: 000|100
  Increase Trend => {CV>THRESHOLD_CV}{TREND>0}{TREND=0}: 010|110
  '''
  dict_current_state_of_each_dev = {}
  end_time_for_current_window = mileston_datetime + datetime.timedelta(minutes = TIME_WINDOW_IN_MINUTES)
  data_extract_from_mileston = data_to_get[(data_to_get.datetime >= mileston_datetime) & (data_to_get.datetime < end_time_for_current_window)].copy(deep = True)
  for dev_id_name in SORTED_LIST_DEVICES_TO_DETECT:

    list_value_of_dev_in_seq = list(data_extract_from_mileston[dev_id_name].values)
    # we have column corresponding to each dev -> present value of each device each time A DEVICE report change value in TIME_WINDOW

    if not list_value_of_dev_in_seq:
      # list_value_of_dev_in_seq is EMPTY LIST => There is no change in ANY device state <=> There NO device report in current TIME_WINDOW
      if is_binary_device_by_id(dev_id_name):
        # Binary device
        dict_current_state_of_each_dev[dev_id_name] = '0'
      else:
        # Numeric device
        dict_current_state_of_each_dev[dev_id_name] = '001'
    else:
      # NUMERIC DEVICE
      # has at least 1 values for this device
      # if has only 1 values -> There is only one device (this device) report change in TIME_WINDOW

      if is_binary_device_by_id(dev_id_name):
        # Binary device
        if 1 in list_value_of_dev_in_seq:
          dict_current_state_of_each_dev[dev_id_name] = '1'
        else:
          dict_current_state_of_each_dev[dev_id_name] = '0'
      else:
        # Numeric device
        temp_state = ''
        trend_value = "ERROR"
        coefficient_variation = "ERROR"
        if len(list_value_of_dev_in_seq) < 3:
          # need >=3 values to detect exact trend and CV

          number_values_need_to_get_from_previous = 0 # init -> not have meaning -> only declare to using outside

          if len(list_value_of_dev_in_seq) == 1:
            number_values_need_to_get_from_previous = 2
          else:
            # 2 values -> there is 2 time this device report
            # cannot 0 value because it pass "if not list_value_of_dev_in_seq:""
            number_values_need_to_get_from_previous = 1


          N_pre_values_from_cur_dev = list(data[(data.datetime < mileston_datetime)].tail(number_values_need_to_get_from_previous)[dev_id_name].values[0:])
          N_pre_values_from_cur_dev.extend(list_value_of_dev_in_seq)
          three_value_of_cur_dev = N_pre_values_from_cur_dev
          # pause(f'IN get_current_state_of_each_dev() function, Lost of data, data has not enough 3 values:{three_value_of_cur_dev} of {dev_id_name}, {mileston_datetime}')

          if len(three_value_of_cur_dev) < 3:
            pause(f'IN get_current_state_of_each_dev() function, Lost of data, data has not enough 3 values:{three_value_of_cur_dev} of {dev_id_name}, {mileston_datetime}')

          trend_value = trenddetector(three_value_of_cur_dev)

          # print(dev_id_name, "three_value_of_cur_dev\n", three_value_of_cur_dev)
          coefficient_variation = coefficient_of_variation(three_value_of_cur_dev)
        else:
          trend_value = trenddetector(list_value_of_dev_in_seq)
          # print(dev_id_name, "list_value_of_dev_in_seq\n", list_value_of_dev_in_seq)
          coefficient_variation = coefficient_of_variation(list_value_of_dev_in_seq)
        if (trend_value == "ERROR") or (coefficient_variation == "ERROR"):
            pause('IN get_current_state_of_each_dev() function, CANNOT set trend_value and coefficient_of_variation')
        else:
          if coefficient_variation > THRESHOLD_CV:
            temp_state += '1'
          else:
            temp_state += '0'
          if trend_value > 0:
            temp_state += '10'
          else:
            if trend_value == 0:
              temp_state += '01'
            else:
              temp_state += '00'

        dict_current_state_of_each_dev[dev_id_name] = temp_state
  return dict_current_state_of_each_dev

In [ ]:
# # get_current_state_of_each_dev(data_valuable.loc[83881].datetime)
# data[data.datetime < data.loc[83881].datetime]

In [ ]:
# get_current_state_of_each_dev(data.loc[83880].datetime), data.loc[83880].datetime

In [ ]:
# # After looking to the source code, I found out the inherit from datetime.datetime which is immutable.
# start = start_time_train
# start = last_time_train
# print(start, start_time_train, data_valuable.iloc[0].datetime)  #2011-06-27 20:51:28.252875 2011-06-26 20:51:28.252875 2011-06-26 20:51:28.252875


start_time_train = data_valuable.iloc[0].datetime
last_time_train = start_time_train + datetime.timedelta(hours = 24)

startTW = start_time_train
while startTW < last_time_train:
  end_TW = startTW + datetime.timedelta(minutes = TIME_WINDOW_IN_MINUTES)
  # print(startTW, end_TW,last_time_train)
  data_TW = data_valuable[(data_valuable.datetime >= startTW) & (data_valuable.datetime < end_TW)].copy(deep = True)

  dict_current_state_of_each_dev = copy.deepcopy(get_current_state_of_each_dev(data_valuable, startTW)) 
  for index_id_each_group in range(len(group_dev)):
    cur_group = dict_get_group_by_id[index_id_each_group]
    cur_group_state = ''
    for device_name in cur_group:
      cur_group_state += dict_current_state_of_each_dev[device_name]

    if cur_group_state not in dict_group_state[index_id_each_group]:
        dict_group_state[index_id_each_group].append(cur_group_state)
  
  startTW = end_TW

In [ ]:
# save_data(dict_group_state, 'dict_group_state_24h_test.data')

In [ ]:
dict_group_state = load_data('dict_group_state_24h_test.data')

In [ ]:
# datetime.datetime(year=2017, month=10, day=24, hour=4, minute=3, second=10, microsecond=7199)

In [ ]:
for i in dict_group_state.keys():
  num_states = len(dict_group_state[i])
  print(i, dict_get_group_by_id.get(i))
  print(num_states)

  max_num_states = 1
  for dev in dict_get_group_by_id.get(i):
    if is_binary_device_by_id(dev):
      max_num_states *= 2
    else:
      max_num_states *= 5
  print(max_num_states)
  if(num_states == max_num_states):
    print("full state")
  print()

print(dict_group_state)

0 ['LS003', 'M011', 'M012', 'MA003', 'MA010', 'MA014']
20
160

1 ['LS002', 'M002', 'MA003']
7
20

2 ['LS015', 'M015', 'MA014']
8
20

3 ['L001', 'LL001', 'LS001', 'LS003', 'M001', 'M002', 'MA003', 'MA009']
26
4000

4 ['LS019', 'LS020']
12
25

5 ['D002', 'LS001', 'LS003', 'M001', 'MA003']
26
200

6 ['LS014', 'M015', 'M016', 'MA014']
14
40

7 ['L005', 'LL005', 'LS018', 'M018', 'M019', 'MA020']
20
400

8 ['LS007', 'LS008']
8
25

9 ['LS010', 'MA010']
7
10

10 ['LS016', 'M016']
8
10

11 ['D006', 'LS007', 'M007', 'M008']
11
40

12 ['M021', 'M022', 'MA020']
6
8

13 ['L002', 'LS003', 'LS009', 'M002', 'MA003', 'MA009']
24
400

14 ['L004', 'LS014', 'LS017', 'LS018', 'M017', 'M018', 'MA014']
48
2000

15 ['L003', 'LS013', 'M018', 'M019', 'MA013', 'MA020']
36
160

16 ['M005', 'M006', 'M007']
8
8
full state

17 ['M019', 'M022', 'MA020']
6
8

18 ['L003', 'LS013', 'LS018', 'M018', 'MA013']
39
200

19 ['LS023', 'MA020', 'MA023']
6
20

20 ['LS010', 'LS012']
9
25

21 ['D005', 'D006', 'M007', 'M008']
5
16


In [ ]:
import numpy as np
#init transition matrix
dict_transition_matrix_by_gr_id = {}
for gr_id in dict_group_state.keys():
    num_states = len(dict_group_state[gr_id])
    dict_transition_matrix_by_gr_id[gr_id] = np.zeros((num_states,num_states))

dict_transition_matrix_by_gr_id

{0: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
       

In [ ]:
# Chu y http://net-informations.com/python/iq/immutable.htm LIST va DICT deu la MUTABLE -> khong de copy luc gan giua 2 list hay hai dict voi nhau la gay
# co loi thi can check can than lai cac lenh lien quan den gan dict, list

start_time_train = data_valuable.iloc[0].datetime
last_time_train = start_time_train + datetime.timedelta(hours = 24)
# print("limit:  ", start_time_train,last_time_train)

# data_train = data_valuable[(data_valuable.datetime >= start_time_train) & (data_valuable.datetime < last_time_train)].copy(deep = True)
# # print("real get", data_train.iloc[0].datetime, data_train.iloc[-1].datetime)


first_time = True
startTW = start_time_train

temp_dict_pre_state_of_each_group = {} #??? co can khong khi no la local nhu duoi co the la du roi-> co gay ra loi gi khong

while startTW < last_time_train:
  end_TW = startTW + datetime.timedelta(minutes = 1)
  print(startTW, end_TW,last_time_train)
  data_TW = data_valuable[(data_valuable.datetime >= startTW) & (data_valuable.datetime < end_TW)]

  temp_dict_cur_state_of_each_group ={}
  dict_current_state_of_each_dev = copy.deepcopy(get_current_state_of_each_dev(data_TW, startTW))
  for index_id_each_group in range(len(group_dev)):
    cur_group = dict_get_group_by_id[index_id_each_group]
    cur_group_state = ''
    for device_name in cur_group:
      cur_group_state += dict_current_state_of_each_dev[device_name]

    temp_dict_cur_state_of_each_group[index_id_each_group] = cur_group_state
  
    if first_time == False:
      # print(first_time)
      # print(index_id_each_group)
      index_state_id_pre =  dict_group_state[index_id_each_group].index(temp_dict_pre_state_of_each_group[index_id_each_group]) 
      index_state_id_cur =  dict_group_state[index_id_each_group].index(temp_dict_cur_state_of_each_group[index_id_each_group])
    
      dict_transition_matrix_by_gr_id[index_id_each_group][index_state_id_pre][index_state_id_cur] += 1

  # temp_dict_pre_state_of_each_group = temp_dict_cur_state_of_each_group.copy()
  temp_dict_pre_state_of_each_group = copy.deepcopy(temp_dict_cur_state_of_each_group)

  if first_time == True:
    first_time = False

  
  startTW = end_TW

2011-06-26 20:51:28.252875 2011-06-26 20:52:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:52:28.252875 2011-06-26 20:53:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:53:28.252875 2011-06-26 20:54:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:54:28.252875 2011-06-26 20:55:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:55:28.252875 2011-06-26 20:56:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:56:28.252875 2011-06-26 20:57:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:57:28.252875 2011-06-26 20:58:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:58:28.252875 2011-06-26 20:59:28.252875 2011-06-27 20:51:28.252875
2011-06-26 20:59:28.252875 2011-06-26 21:00:28.252875 2011-06-27 20:51:28.252875
2011-06-26 21:00:28.252875 2011-06-26 21:01:28.252875 2011-06-27 20:51:28.252875
2011-06-26 21:01:28.252875 2011-06-26 21:02:28.252875 2011-06-27 20:51:28.252875
2011-06-26 21:02:28.252875 2011-06-26 21:03:28.252875 2011-06-27 20:51:28.252875
2011-06-26 21:03:28.252875 2

In [ ]:
# print(dict_transition_matrix_by_gr_id[0].shape, np.sum( dict_transition_matrix_by_gr_id[0] ==  0),np.sum( dict_transition_matrix_by_gr_id[0] !=  0))
set_dev_can_be_detect = set()
print('Existed state of cur group and group transition state:\ngroup_id - group - existed_state\nshape - existed state\n')
for gr_id in dict_transition_matrix_by_gr_id.keys():
  
  num_states = len(dict_group_state[gr_id])
  print(f'{gr_id} - {dict_get_group_by_id.get(gr_id)} - {num_states}/{2**len(list(dict_group_state[gr_id])[0])}')
  
  existed = np.sum(dict_transition_matrix_by_gr_id[gr_id] !=  0)
  not_existed = np.sum( dict_transition_matrix_by_gr_id[gr_id] ==  0)
  print(f'{dict_transition_matrix_by_gr_id[gr_id].shape} - {existed}/{existed + not_existed}\n')
  if (existed != (existed + not_existed)) or  (num_states != (2**len(list(dict_group_state[gr_id])[0])) ):
    set_dev_can_be_detect.update(dict_get_group_by_id.get(gr_id))
print(f"num of dev can be detect anomaly: {len(set_dev_can_be_detect)}\n {set_dev_can_be_detect}" )

Existed state of cur group and group transition state:
group_id - group - existed_state
shape - existed state

0 - ['LS003', 'M011', 'M012', 'MA003', 'MA010', 'MA014'] - 20/256
(20, 20) - 66/400

1 - ['LS002', 'M002', 'MA003'] - 7/32
(7, 7) - 27/49

2 - ['LS015', 'M015', 'MA014'] - 8/32
(8, 8) - 24/64

3 - ['L001', 'LL001', 'LS001', 'LS003', 'M001', 'M002', 'MA003', 'MA009'] - 26/16384
(26, 26) - 71/676

4 - ['LS019', 'LS020'] - 12/64
(12, 12) - 34/144

5 - ['D002', 'LS001', 'LS003', 'M001', 'MA003'] - 26/512
(26, 26) - 71/676

6 - ['LS014', 'M015', 'M016', 'MA014'] - 14/64
(14, 14) - 36/196

7 - ['L005', 'LL005', 'LS018', 'M018', 'M019', 'MA020'] - 20/1024
(20, 20) - 59/400

8 - ['LS007', 'LS008'] - 8/64
(8, 8) - 23/64

9 - ['LS010', 'MA010'] - 7/16
(7, 7) - 16/49

10 - ['LS016', 'M016'] - 8/16
(8, 8) - 20/64

11 - ['D006', 'LS007', 'M007', 'M008'] - 11/64
(11, 11) - 37/121

12 - ['M021', 'M022', 'MA020'] - 6/8
(6, 6) - 26/36

13 - ['L002', 'LS003', 'LS009', 'M002', 'MA003', 'MA009'] 

In [ ]:
def get_error_start_pos(data_test_to_add_fault, fault_dev):
  success = True
  list_origin_index_of_fault_dev = list(data_test_to_add_fault[data_test_to_add_fault['device_id'] == fault_dev].index.values)
  len_list_origin_index_of_fault_dev = len(list_origin_index_of_fault_dev)

  if(len_list_origin_index_of_fault_dev < 25):
    success = False
    # not enough value to make fault
    return success, -1
  index_first_time_fault_occur = random.choice(list_origin_index_of_fault_dev[5:10])
  return success, index_first_time_fault_occur

# get device list  make fault to detect
list_dev_data_test = list(set_dev_can_be_detect.intersection(set(data_valuable.device_id.unique())))

num_dev_data_test = len(list_dev_data_test)
# print(num_dev_data_test, list_dev_data_test)

bool_binary_device = (data_valuable.is_binary_device == True)
list_numeric_device = list(data_valuable[(~bool_binary_device)].device_id.unique()).copy()
list_numeric_device = list(set_devices_to_detect.intersection(set(list_numeric_device)))
len_numeric_device = len(list_numeric_device)

In [ ]:
fault_name_by_index = {0: 'Fail-stop', 1: 'Stuck-at', 2: 'High variance', 3:'Spike'}

def gen_fault(df_gen_fault):
  data_have_fault = df_gen_fault.copy(deep=True)
  type_of_gen_fault = random.randint(0,3)
  '''
  type of fault:

      0: Fail stop :
      Choose random device ( in device list existed in data_test)
      Remove all data of this device
      Both numeric and binary device
      1: Stuck-at: Both numeric and binary dev
      2: High-Variance:  -> for numeric dev 
      3: Spike: -> for numeric dev

  '''
  fault_dev = ''
  fault_name = ''
  index_first_time_fault_occur = 0
  success = True

  if (type_of_gen_fault == 0) or (type_of_gen_fault == 1):
    fault_name = fault_name_by_index[type_of_gen_fault]
    fault_dev = list_dev_data_test[random.randint(0,num_dev_data_test-1)]

    if data_have_fault[data_have_fault['device_id'] == fault_dev].empty:
      success = False
    else:
      success, index_first_time_fault_occur = get_error_start_pos(data_have_fault, fault_dev)
      if success:
        # print(data_have_fault)
        date_time_first_time_fault_occur =  data_have_fault.loc[index_first_time_fault_occur].datetime
        value_first_time_fault_occur = data_have_fault.loc[index_first_time_fault_occur].device_value

        data_have_fault.loc[(data_have_fault['datetime'] > date_time_first_time_fault_occur), fault_dev] = value_first_time_fault_occur
        # print(data_have_fault)

        data_have_fault.drop(data_have_fault[(data_have_fault['datetime'] > date_time_first_time_fault_occur) & (data_have_fault['device_id'] == fault_dev)].index, axis =0, inplace = True)
        # print(data_have_fault)
        # print(fault_dev,data_have_fault, "RESULT", data_have_fault[data_have_fault.device_id == fault_dev])

  elif type_of_gen_fault == 2:
    # only for numeric device
    # HIGH VARIANCE
    fault_name = fault_name_by_index[type_of_gen_fault]
    fault_dev = list_numeric_device[random.randint(0,len_numeric_device-1)]

    if data_have_fault[data_have_fault['device_id'] == fault_dev].empty:
      success = False
    else:
      success, index_first_time_fault_occur = get_error_start_pos(data_have_fault, fault_dev)
      if success:
        # old values

        date_time_first_time_fault_occur =  data_have_fault.loc[index_first_time_fault_occur].datetime
        origin_value_first_time_fault_occur = data_have_fault.loc[index_first_time_fault_occur].device_value

        list_value = list(data_have_fault.loc[((data_have_fault['device_id'] == fault_dev) & (data_have_fault['datetime'] > date_time_first_time_fault_occur)), 'device_value'].values).copy()
        len_list_value = len(list_value)

        value_min = df_iqr[df_iqr.device_id == fault_dev]._min.values[0]
        value_max = df_iqr[df_iqr.device_id == fault_dev]._max.values[0]
        value_mean = df_iqr[df_iqr.device_id == fault_dev]._mean.values[0]

        if origin_value_first_time_fault_occur > value_mean:
          pre_value = value_min
        else:
          pre_value = value_max

        data_have_fault.loc[index_first_time_fault_occur, 'device_value'] = pre_value

        # make new value 
        for i in range(0,len_list_value):
          if pre_value > value_mean:
            list_value[i] = value_min
            pre_value = list_value[i]
          else:
            list_value[i] = value_max
            pre_value = list_value[i]

        data_have_fault.loc[(data_have_fault['datetime'] >= date_time_first_time_fault_occur), fault_dev] = np.nan
        data_have_fault.loc[((data_have_fault['device_id'] == fault_dev) & (data_have_fault['datetime'] > date_time_first_time_fault_occur)), 'device_value'] = list_value

        data_have_fault.loc[((data_have_fault.device_id == fault_dev) & (data_have_fault['datetime'] >=date_time_first_time_fault_occur)), fault_dev] = data_have_fault.device_value
        data_have_fault.ffill(inplace=True)

  else:
    # only for numeric device
    # SPKIE
    fault_name = fault_name_by_index[type_of_gen_fault]
    fault_dev = list_numeric_device[random.randint(0,len_numeric_device-1)]

    if data_have_fault[data_have_fault['device_id'] == fault_dev].empty:
      success = False
    else:
      success, index_first_time_fault_occur = get_error_start_pos(data_have_fault, fault_dev)
      if success:
        # old values

        date_time_first_time_fault_occur =  data_have_fault.loc[index_first_time_fault_occur].datetime

        list_value = list(data_have_fault.loc[((data_have_fault['device_id'] == fault_dev) & (data_have_fault['datetime'] > date_time_first_time_fault_occur)), 'device_value'].values).copy()
        len_list_value = len(list_value)
        print(list_value, len_list_value)

        if len_list_value <10:
          print('not enough data to make fault')
          success = False
        else:
          number_of_fault = len_list_value*20//100 #(20%)

          value_min = df_iqr[df_iqr.device_id == fault_dev]._min.values[0]
          value_max = df_iqr[df_iqr.device_id == fault_dev]._max.values[0]
          
          list_value = [value_min]*len_list_value
          # print(f"new list {len(list_value)}: ", list_value)
          milestone_pos_spike = (len_list_value-number_of_fault)//(number_of_fault+1)
          pos_spike = milestone_pos_spike
          for cnt in range(number_of_fault):
            # print("pos spike", pos_spike)
            list_value[pos_spike] = value_max
            pos_spike = pos_spike + milestone_pos_spike + 1



          data_have_fault.loc[(data_have_fault['datetime'] > date_time_first_time_fault_occur), fault_dev] = np.nan

          data_have_fault.loc[((data_have_fault['device_id'] == fault_dev) & (data_have_fault['datetime'] > date_time_first_time_fault_occur)), 'device_value'] = list_value

          data_have_fault.loc[((data_have_fault.device_id == fault_dev) & (data_have_fault['datetime'] >=date_time_first_time_fault_occur)), fault_dev] = data_have_fault.device_value

          data_have_fault.ffill(inplace=True)

  return success, fault_name, fault_dev, index_first_time_fault_occur, data_have_fault

In [ ]:
# start_TEST = data_valuable.iloc[0].datetime

# data_TEST = data_valuable[data_valuable.datetime <= (start_TEST + datetime.timedelta(hours=24))].copy(deep = True)
# print(data_TEST)

                        datetime device_id  device_value  ...  MA014  MA020  MA023
83881 2011-06-26 20:51:28.252875      L001             1  ...    0.0    0.0    0.0
83882 2011-06-26 20:51:28.427867     LL001            63  ...    0.0    0.0    0.0
83883 2011-06-26 20:51:28.929415     LS003             2  ...    0.0    0.0    0.0
83884 2011-06-26 20:51:28.959199     MA003             1  ...    0.0    0.0    0.0
83885 2011-06-26 20:51:29.282708     LS001             7  ...    0.0    0.0    0.0
...                          ...       ...           ...  ...    ...    ...    ...
94061 2011-06-27 20:51:23.946719     LS007             2  ...    0.0    0.0    0.0
94062 2011-06-27 20:51:23.988094      M007             1  ...    0.0    0.0    0.0
94063 2011-06-27 20:51:24.467465      M005             0  ...    0.0    0.0    0.0
94064 2011-06-27 20:51:26.096992      M008             1  ...    0.0    0.0    0.0
94065 2011-06-27 20:51:26.572982      M007             0  ...    0.0    0.0    0.0

[10

In [ ]:
# # CHECK XEM SINH LOI DUNG CHUA
# success, fault_name, fault_dev, index_first_time_fault_occur, data_have_fault = gen_fault(data_TEST)

# print(success, fault_name, fault_dev, index_first_time_fault_occur, data_have_fault)

# print(data_have_fault[(data_have_fault.datetime >= data_have_fault.loc[index_first_time_fault_occur].datetime)][['device_id', 'device_value', fault_dev]])
# print(data_have_fault[(data_have_fault.device_id == fault_dev) & (data_have_fault.datetime >= data_have_fault.loc[index_first_time_fault_occur].datetime)][['device_id', 'device_value', fault_dev]].head(20))

In [ ]:
# Return the Hamming distance between string1 and string2.
# string1 and string2 should be the same length.
def hamming_distance(string1, string2): 
    # Start with a distance of zero, and count up
    list_diff_pos = []
    distance = 0
    # Loop over the indices of the string
    L = len(string1)
    for i in range(L):
        # Add 1 to the distance if these two characters are not equal
        if string1[i] != string2[i]:
            distance += 1
            list_diff_pos.append(i)
    # Return the final count of differences
    return list_diff_pos, distance

def get_diff_bits_pos(string1, list_string):
  list_diff_pos, min_dis = hamming_distance(string1, list_string[0])
  for i in range(1, len(list_string)):
    curr_diff_pos, curr_dis  = hamming_distance(string1, list_string[i])
    if curr_dis < min_dis:
      min_dis = curr_dis
      list_diff_pos = curr_diff_pos
    elif curr_dis == min_dis:
      list_diff_pos.extend(curr_diff_pos)
      list_diff_pos = list(set(list_diff_pos))
  return list_diff_pos, min_dis

In [ ]:
def detect_anomaly(data_fault, _hours = 24):
  suggest_exact_from_CRE_CH = []
  suggest_list_group_CRE_CH = []
  suggest_list_group_TS_CH = []


  df_copy = data_fault.copy(deep=True)

  start_time_TEST = df_copy.iloc[0].datetime
  last_time_TEST = start_time_TEST + datetime.timedelta(days = 1)

  first_time = True
  startTW = start_time_TEST
  temp_dict_pre_state_of_each_group = {} #??? co can khong khi no la local nhu duoi co the la du roi-> co gay ra loi gi khong

  while startTW < last_time_TEST:
    end_TW = startTW + datetime.timedelta(minutes = 1)
    # print(startTW, end_TW,last_time_TEST)
    data_TW = df_copy[(df_copy.datetime >= startTW) & (df_copy.datetime < end_TW)]

    temp_dict_cur_state_of_each_group ={}
    dict_current_state_of_each_dev = copy.deepcopy(get_current_state_of_each_dev(df_copy, startTW))
    for index_id_each_group in range(len(group_dev)):
      cur_group = dict_get_group_by_id[index_id_each_group]
      cur_group_state = ''
      for device_name in cur_group:
        cur_group_state += dict_current_state_of_each_dev[device_name]
      
      temp_dict_cur_state_of_each_group[index_id_each_group] = cur_group_state
    
    for index_id_each_group in range(len(group_dev)):
      if temp_dict_cur_state_of_each_group[index_id_each_group] not in dict_group_state[index_id_each_group]:
        suggest_list_group_CRE_CH.append(dict_get_group_by_id.get(index_id_each_group))

        bit_diff_pos, min_dis = get_diff_bits_pos(temp_dict_cur_state_of_each_group[index_id_each_group], dict_group_state[index_id_each_group])
          # print(min_dis, bit_diff_pos)
        for pos in bit_diff_pos:
          gr_device = dict_get_group_by_id.get(index_id_each_group)
          max_index = -1;
          for dev in gr_device:
            if is_binary_device_by_id(dev):
              max_index = max_index + 1
            else:
              max_index = max_index + 3
            if pos <= max_index:
              suggest_exact_from_CRE_CH.append(dev)
              break
          # pause("0")
      else:
        if first_time == False:
        # print(first_time)
        # print(index_id_each_group)
          if temp_dict_pre_state_of_each_group[index_id_each_group] in dict_group_state[index_id_each_group]:
            index_state_id_pre =  dict_group_state[index_id_each_group].index(temp_dict_pre_state_of_each_group[index_id_each_group])
            index_state_id_cur =  dict_group_state[index_id_each_group].index(temp_dict_cur_state_of_each_group[index_id_each_group])

            if dict_transition_matrix_by_gr_id[index_id_each_group][index_state_id_pre][index_state_id_cur] == 0:
                suggest_list_group_TS_CH.append(dict_get_group_by_id.get(gr_id))
                # pause("1")

    temp_dict_pre_state_of_each_group = copy.deepcopy(temp_dict_cur_state_of_each_group)
    if first_time == True:
      first_time = False
    startTW = end_TW

  return suggest_exact_from_CRE_CH, suggest_list_group_CRE_CH, suggest_list_group_TS_CH

In [ ]:
start_TEST = data_valuable.iloc[0].datetime
end_TEST = start_TEST + datetime.timedelta(hours=24)
data_TEST = data_valuable[(data_valuable.datetime < end_TEST) & (data_valuable.datetime >= start_TEST)].copy(deep = True)
# print(data_TEST)



numberofsuccess = 0

cnt = 0
while True:
  if cnt >9:
    break;
  success, fault_name, fault_dev, index_first_time_fault_occur, data_have_fault = gen_fault(data_TEST)

  if success:

    print(success, fault_name, fault_dev, index_first_time_fault_occur, data_have_fault)

    print(data_have_fault[(data_have_fault.datetime >= data_have_fault.loc[index_first_time_fault_occur].datetime)][['device_id', 'device_value', fault_dev]])
    print(data_have_fault[(data_have_fault.device_id == fault_dev) & (data_have_fault.datetime >= data_have_fault.loc[index_first_time_fault_occur].datetime)][['device_id', 'device_value', fault_dev]].head(20))
    cnt = cnt+1
    print(fault_name, fault_dev)
    list_suggest_exact_from_CRE_CH, list_suggest_list_group_CRE_CH, list_suggest_list_group_TS_CH = detect_anomaly(data_have_fault, 24)
    print('list_suggest_exact_from_CRE_CH:')
    print(list_suggest_exact_from_CRE_CH)


    _list_suggest_list_group_CRE_CH = []
    print('list_suggest_list_group_CRE_CH:')
    for ele in list_suggest_list_group_CRE_CH:
      if ele not in _list_suggest_list_group_CRE_CH:
        _list_suggest_list_group_CRE_CH.append(ele)
        print(ele, list_suggest_list_group_CRE_CH.count(ele))
    print(_list_suggest_list_group_CRE_CH)

    _list_suggest_list_group_TS_CH = []
    print('list_suggest_list_group_TS_CH:')
    for ele in list_suggest_list_group_TS_CH:
      if ele not in _list_suggest_list_group_TS_CH:
        _list_suggest_list_group_TS_CH.append(ele)
        print(ele, list_suggest_list_group_TS_CH.count(ele))
    print(_list_suggest_list_group_TS_CH)


    success_from_suggest_list_group_CRE_CH = False
    for lst in _list_suggest_list_group_CRE_CH:
      if fault_dev in lst:
          success_from_suggest_list_group_CRE_CH = True

    success_from_suggest_list_group_TS_CH = False 
    for lst in _list_suggest_list_group_TS_CH:
      if fault_dev in lst:
          success_from_suggest_list_group_TS_CH = True
    
    if success_from_suggest_list_group_CRE_CH or success_from_suggest_list_group_TS_CH:
      numberofsuccess += 1


  
print(numberofsuccess)

True Stuck-at LS010 84935                         datetime device_id  device_value  ...  MA014  MA020  MA023
83881 2011-06-26 20:51:28.252875      L001             1  ...    0.0    0.0    0.0
83882 2011-06-26 20:51:28.427867     LL001            63  ...    0.0    0.0    0.0
83883 2011-06-26 20:51:28.929415     LS003             2  ...    0.0    0.0    0.0
83884 2011-06-26 20:51:28.959199     MA003             1  ...    0.0    0.0    0.0
83885 2011-06-26 20:51:29.282708     LS001             7  ...    0.0    0.0    0.0
...                          ...       ...           ...  ...    ...    ...    ...
94061 2011-06-27 20:51:23.946719     LS007             2  ...    0.0    0.0    0.0
94062 2011-06-27 20:51:23.988094      M007             1  ...    0.0    0.0    0.0
94063 2011-06-27 20:51:24.467465      M005             0  ...    0.0    0.0    0.0
94064 2011-06-27 20:51:26.096992      M008             1  ...    0.0    0.0    0.0
94065 2011-06-27 20:51:26.572982      M007             0  ...

In [ ]:
lst_Vss = [2]*10
lst_Vss

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [ ]:
dictds = {}
dictds[0] = ['MVD','d']
gr = dictds[0]
print(gr)
gr = ['SDF', 'dfsdf']
print(gr, dictds)


['MVD', 'd']
['SDF', 'dfsdf'] {0: ['MVD', 'd']}


In [ ]:
dictds = {}

a = ''
a +='1'
a += '0'
dictds[0] = a
print(a, dictds[0])

a = 'dfsdf'
# a +='0'
# a += '1'
# dictds[0] = a
print(a, dictds[0])


10 10
dfsdf 10


In [ ]:
d = {'datetime': [1,2,3,4,5,6,7], 'device': ['M2', 'M1', 'M2', 'M3', 'M1','M2','M1'], 'valueLL': [0, -1, 2,3, 4,5, 6], 'M1': [-10, -1,-1,-1,4,4,6], 'M2': [0,0,2,2,2,5,5]}
dfsss = pd.DataFrame(data=d)
print(dfsss)
# collname = 'col2'
# dfsss[dfsss.col1 <= 2].tail(3)[collname].values[0:3], list(dfsss[dfsss.col1 <= -2][collname].values), not list(dfsss[dfsss.col1 <= -2][collname].values)

dfsss.valueLL

   datetime device  valueLL  M1  M2
0         1     M2        0 -10   0
1         2     M1       -1  -1   0
2         3     M2        2  -1   2
3         4     M3        3  -1   2
4         5     M1        4   4   2
5         6     M2        5   4   5
6         7     M1        6   6   5


0    0
1   -1
2    2
3    3
4    4
5    5
6    6
Name: valueLL, dtype: int64

In [ ]:
# date_time_first =  dfsss.loc[1].datetime
# fd = 'M1'
# value_first_time = dfsss.loc[1].valueLL
# print('value_first_time', value_first_time)
# dfsss.loc[(dfsss['datetime'] > date_time_first), fd] = value_first_time
# print(dfsss)
# dfsss.drop(dfsss[(dfsss['datetime'] > date_time_first) & (dfsss['device'] == fd)].index, axis =0, inplace = True)
# print(dfsss)

In [ ]:
print(dfsss)
date_time_first =  dfsss.loc[1].datetime
fd = 'M1'

list_vl = list(dfsss.loc[((dfsss['device'] == fd) & (dfsss['datetime'] > date_time_first)), 'valueLL'].values).copy()
len_list_vl = len(list_vl)

vl_min = -100
vl_max = 100
vl_mean = 0
origin_occur = dfsss.loc[1].valueLL

if origin_occur > vl_mean:
  pre_vl = vl_min
else:
  pre_vl = vl_max

dfsss.loc[1, 'valueLL'] = pre_vl

# make new value 
for i in range(0,len_list_vl):
  if pre_vl > vl_mean:
    list_vl[i] = vl_min
    pre_vl = list_vl[i]
  else:
    list_vl[i] = vl_max
    pre_vl = list_vl[i]

dfsss.loc[(dfsss['datetime'] >= date_time_first), fd] = np.nan
dfsss.loc[((dfsss['device'] == fd) & (dfsss['datetime'] >date_time_first)), 'valueLL'] = list_vl

dfsss.loc[((dfsss.device == fd) & (dfsss['datetime'] >=date_time_first)), fd] = dfsss.valueLL


dfsss.ffill(inplace=True)

dfsss

   datetime device  valueLL     M1  M2
0         1     M2        0  -10.0   0
1         2     M1     -100 -100.0   0
2         3     M2        2    NaN   2
3         4     M3        3    NaN   2
4         5     M1      100  100.0   2
5         6     M2        5    NaN   5
6         7     M1     -100 -100.0   5


,datetime,device,valueLL,M1,M2
0,1,M2,0,-10.0,0
1,2,M1,100,100.0,0
2,3,M2,2,100.0,2
3,4,M3,3,100.0,2
4,5,M1,-100,-100.0,2
5,6,M2,5,-100.0,5
6,7,M1,100,100.0,5


In [ ]:
# coefficient_of_variation([1,1,20])

1.2213662584131277

In [ ]:
assss = 2
bssss = 3
num_sssss = "ERROR"
print(num_sssss)
if assss == 2:
  num_sssss = 0
  if bssss == 3:
    num_sssss = 1
  else:
    num_sssss = 2
  print(num_sssss)
print(num_sssss)

ERROR
1
1


In [ ]:
myDict = {'one': ['1', '2', '3']}
myDict2 = myDict.copy()
print(myDict)
print(myDict2)
myDict['one'] = 3
print(myDict)
myDict2

{'one': ['1', '2', '3']}
{'one': ['1', '2', '3']}
{'one': 3}


{'one': ['1', '2', '3']}

In [ ]:
df_testd = df_testd.assign(**dict_T)

In [ ]:
ss = data.datetime.iloc[0]
ss

In [ ]:
xx = datetime.datetime(2011, 6, 16)
print(xx) 

In [ ]:
xx - ss

In [ ]:
data.datetime.iloc[0].value

In [ ]:
ls = []
ls.append(1)
ls.append(2)
ls
df_testd = pd.DataFrame(columns=['A', 'B'])
df_testd.loc[0] = [9,2]
df_testd.loc[3] = [1,10]
df_testd.loc[2] = [9,1]

df_testd.iloc[1]

keys = ['E', 'F']
values =[np.nan]*2
dict_T = dict(zip(keys, values))
dict_T


{'E': nan, 'F': nan}

In [ ]:
df_testd = df_testd.assign(**dict_T)

In [ ]:
df_testd

,A,B,E,F
0,9,2,NaN,NaN
3,1,10,NaN,NaN
2,9,1,NaN,NaN


In [ ]:
df_testd.loc[df_testd['A'] == 9,'E'] = df_testd['B']

In [ ]:
df_testd

,A,B,E,F
0,9,2,2,NaN
3,1,10,2,10.0
2,9,1,1,10.0
4,9,1,1,15.0


In [ ]:
df_testd_2 = df_testd[df_testd.A == 9]
df_testd_2

,A,B,E,F
0,9,2,2,NaN
2,9,1,1,NaN


In [ ]:
df_testd.loc[3,'F']= 10
df_testd.loc[4,'F'] =15
df_testd

,A,B,E,F
0,9,2,2,NaN
3,1,10,NaN,10.0
2,9,1,1,NaN
4,NaN,NaN,NaN,15.0


In [ ]:
df_testd.notna()

,A,B,E,F
0,True,True,True,False
3,True,True,False,True
2,True,True,True,False
4,False,False,False,True


In [ ]:
df_testd.fillna(method='bfill', axis=1).iloc[:, 0]


0     9.0
3     1.0
2     9.0
4    15.0
Name: A, dtype: float64

In [ ]:
df_testd = df_testd.ffill()
df_testd.loc[2,'A'] = np.nan
df_testd

,A,B,E,F
0,9.0,2,2,NaN
3,1.0,10,2,10.0
2,NaN,1,1,10.0
4,9.0,1,1,15.0


In [ ]:
df_testd[['F']].first_valid_index()

3

0
0
0
3


In [ ]:
df_testd_2.iloc[0]

A      9
B      2
E      2
F    NaN
Name: 0, dtype: object

In [ ]:
df_testd_reset_index = df_testd.reset_index()
df_testd_reset_index

,index,A,B,E,F
0,0,2,3,NaN,NaN
1,3,9,10,HELP,NaN
2,2,9,10,HELP,NaN


In [ ]:
list(df.columns)

['A', 'B', 'C']

In [ ]:
ss0 = [3,  4,  1]
ss00 = np.arange(len(ss0))
print(type(ss00), ss00)
ss = np.array(ss0)
print(type(ss), ss)
trenddetector(ss0), trenddetector(ss0)== -1